#### 1. Import necessary library

In [1]:
conda install tensorflow

In [3]:
#!pip3 install tensorflow-gpu==2.0.0-rc0 
!pip install tensorflow
!pip install keras
!pip install pillow

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import pickle
import os

In [7]:
from sklearn.preprocessing import LabelBinarizer

In [8]:
import wget

ModuleNotFoundError: No module named 'wget'

#### 2. Download data

In [ ]:
# http://www.laurencemoroney.com/horses-or-humans-dataset/
training = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
validation = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

def bar_custom(current, total, width=80):
    print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))

In [ ]:
wget.download(training, bar=bar_custom)
wget.download(validation, bar=bar_custom)

Create folder data

In [11]:
data_folder = "./data"
training_data_folder = data_folder +   '/training'
validation_data_folder = data_folder + '/validation'

In [ ]:
def create_folder(folder):
    if not os.path.isdir(folder):
        print('new directry has been created')
        os.system('mkdir {}'.format(folder))

create_folder(data_folder)

Unzip data

In [ ]:
!unzip -q horse-or-human.zip -d $training_data_folder
#bug when upzip, training_data_folder added '\r' behind => os.listdir()

!unzip -q validation-horse-or-human.zip -d $validation_data_folder

replace ./data/validation/horses/horse1-000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#### 3. Read data

In [9]:
train_generator = ImageDataGenerator(rescale=1/255)
valid_generator = ImageDataGenerator(rescale=1/255)

In [12]:
generated_train_data = train_generator.flow_from_directory(
    training_data_folder,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [13]:
generated_valid_data = valid_generator.flow_from_directory(
    validation_data_folder,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


#### 4. Build model

In [22]:
model = Sequential()

In [23]:
model.add(Conv2D(16, (3 , 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

In [24]:
model.add(Conv2D(32, (3 , 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3 , 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [25]:
model.add(Flatten())

In [26]:
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.summary()

In [28]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['acc'])

#### 5. Train model

In [29]:
history = model.fit_generator(
    generated_train_data,
    steps_per_epoch = 8,
    epochs = 15,
    verbose = 1,
    validation_data = generated_valid_data,
    validation_steps = 8
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 8 steps, validate for 8 steps
Epoch 1/15
8/8 [==============================] - 7s 877ms/step - loss: 1.6204 - acc: 0.6685 - val_loss: 0.1152 - val_acc: 0.8203


#### 6. Predict